In [1]:
from os import walk
from joblib import Parallel, delayed
import librosa
import python_speech_features
import numpy as np
import multiprocessing
import json
from deep_audio import Directory, Audio, JSON

In [2]:
num_cores = multiprocessing.cpu_count()

sampling_rate = 24000

path = f'audios/{sampling_rate}'

f = Directory.filenames(path)

data = {
    "mapping": [],
    "mfcc": [],
    "labels": []
}

In [3]:
def process_directory(dir, index, library):
    signal, sr = Audio.read(f'{path}/{dir}', sr=sampling_rate, normalize=True)

    signal = np.array(signal)

    signal = signal[:len(signal) - len(signal) % (sr * 5)]

    segments = len(signal) // (sr * 5)

    m = {
        "mfcc": [],
        "labels": [index] * segments
    }

    for i in range(segments):
        start_sample = sr * i * 5
        finish_sample = start_sample + (sr * 5)

        sample = signal[start_sample:finish_sample]

        n_mfcc = 13
        n_mels = 40
        n_fft = 2048 
        hop_length = 512
        fmin = 0
        fmax = None

        if library == 'librosa':
            mfcc = librosa.feature.mfcc(y=sample, sr=sr, n_fft=n_fft,
                                        n_mfcc=n_mfcc, n_mels=n_mels,
                                        hop_length=hop_length,
                                        fmin=fmin, fmax=fmax, htk=False)
        elif library == 'psf':
            mfcc = python_speech_features.mfcc(signal=sample, samplerate=sr, winlen=n_fft / sr, winstep=hop_length / sr,
                                            numcep=n_mfcc, nfilt=n_mels, nfft=n_fft, lowfreq=fmin, highfreq=fmax,
                                            preemph=0.0, ceplifter=0, appendEnergy=False)

        elif library == 'torchaudio':
            pass
        
        elif library == 'spafe':
            pass

        m['mfcc'].append(mfcc.tolist())

    print(f'{dir} -> segments: {segments}')
    return m

In [4]:
def object_mfcc_to_json(m, library):
    data['mapping'] = [file.replace('.wav', '') for file in f]

    for i in m:
        data['mfcc'].extend(i['mfcc'])
        data['labels'].extend(i['labels'])

    JSON.create_json_file(f'processed/mfcc/{library}/mfcc_{sampling_rate}.json', data)


In [5]:
if __name__ == '__main__':
    for library in ['librosa', 'psf']:
        m = Parallel(n_jobs=num_cores//2, verbose=len(f), temp_folder='./tmp/')(
            delayed(process_directory)(i, j, library) for j, i in enumerate(f))
        object_mfcc_to_json(m, library)
        data = {
            "mapping": [],
            "mfcc": [],
            "labels": []
        }


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   14.9s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   15.7s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   16.2s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   16.4s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   16.6s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:   16.9s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:   17.0s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:   18.4s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   21.2s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:   22.0s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:   22.6s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:   23.5s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:   24.0s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:   24.6s
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:   26.0s
[Parallel(